# 9장. 범주형
범주형 데이터는 두 종류의 값을 플래그 값, 자료형을 불리언(boolean)형이라고 합니다. <br>
불리언형은 프로그램에서 True 또는 False값으로 나타냅니다.



---------------------------------------------------------------
## 9.1 범주형으로 변환
* 고객 테이블의 성별을 불리언형과 범주형으로 변환

In [1]:
import pandas as pd
import numpy as np

In [6]:
customer_tb=pd.read_csv('./data/customer.csv',encoding='UTF-8')

In [5]:
customer_tb

,customer_id,age,sex,home_latitude,home_longitude
0,c_1,41,man,35.092193,136.512347
1,c_2,38,man,35.325076,139.410551
2,c_3,49,woman,35.120543,136.511179
3,c_4,43,man,43.034868,141.240314
4,c_5,31,man,35.102661,136.523797
...,...,...,...,...,...
995,c_996,44,man,34.465648,135.373787
996,c_997,35,man,35.345372,139.413754
997,c_998,32,woman,43.062267,141.272126
998,c_999,48,woman,38.172800,140.464198


In [8]:
# sex가 man일때 TRUE값인 불리언형을 추가한다.
## astype이 아니여도 불리언형으로 변환한다.
customer_tb[['sex_is_man']]=(customer_tb[['sex']]=='man').astype('bool')

In [22]:
# sex를 범주형으로 변환한다.
customer_tb['sex_c']=pd.Categorical(customer_tb['sex'],categories=['man','woman'])    # categories매개변수에 마스터 데이터의 배열을 지정 가능.

In [23]:
(customer_tb['sex_c']).dtype

CategoricalDtype(categories=['man', 'woman'], ordered=False)

In [24]:
# astype함수로도 변환할 수 있다.
customer_tb['sex_c']=customer_tb['sex_c'].astype('category')

In [29]:
# 인덱스 데이터는 codes에 저장된다.
customer_tb['sex_c'].cat.codes

0      0
1      0
2      1
3      0
4      0
      ..
995    0
996    0
997    1
998    1
999    0
Length: 1000, dtype: int8

In [30]:
# 마스터 데이터는 categories에 저장된다.
customer_tb['sex_c'].cat.categories

Index(['man', 'woman'], dtype='object')

---------------------------------------------------------------
## 9.2 더미 변수화
* 더미 변수는 범주의 종류 수만큼 생성한다.
* 머신러닝에서 예측 모델을 만들 때 더미변수를 하나 줄일 수 있어 필요한 학습 데이터를 조금 줄일 수 있지만, <br>
이 방법을 사용하지 않는 편이 좋을 때도 있다.
    * ex) 더미변수를 줄이지 않고 분석하면 더미 변수의 중요도(중회귀 모델의 각 더미 변수의 계수)로 연령대의 이용 금액이 어떠한 영향을 받는지 알 수 있다. <br>
    하지만 더미 변수를 하나 줄이면 남은 더미 변수에 반영되므로 각 연령대의 이용 금액이 어떠한 영향을 받는지 파악하기 어려움.
    
 * 이처럼 각 변줏값의 영향을 분석하고 싶을 때 더미 변수를 줄이는 방법은 좋지 않다.

#### Q1 : 고객 테이블의 성별을 더미 변수로 만들어보자.
* man이면 1, woman 이면 0

In [10]:
# 더미 변수로 만들기 전 범주형으로 변환한다.
customer_tb['sex']=pd.Categorical(customer_tb['sex'])

In [15]:
# drop_first를 False로 하면 범줏값의 모든 종류 값의 더미 플래그를 생성한다.
dummy_vars=pd.get_dummies(customer_tb['sex'],drop_first=False)

In [17]:
dummy_vars.head()

,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


---------------------------------------------------------------
## 9.3 범줏값의 집약
범주형을 사용할 때 해당하는 데이터 수가 적은 범줏값이 존재하면, 적은 데이터에서 범줏값의 특성을 학습하게 되므로 과학습이 발생하기 쉽다. <br>
또한 애드폭 분석을 실행할 때 범줏값 종류가 많아지면 집계가 어렵다. <br>
이런 문제가 발생하지 않도록 데이터 수가 극단적으로 적은 범줏값을 다른 범줏값과 묶는 방법이 있다. 

> add_categories() 함수 <br>
* 범주형 마스터 데이터를 추가하는 함수

> remove_unused_categories() 함수
* 범주형 마스터 데이터를 제거하는 함수

#### Q1 : 고객 테이블의 연령을 10세 단위로 구분하여 범주형으로 변환하고, 60세 이상인 경우엔 '60세 이상' 이라는 범줏값으로 변환하자.

In [29]:
# 범주형으로 변환
customer_tb['age_rank']=pd.Categorical(np.floor(customer_tb['age']/10)*10)

In [24]:
np.floor(customer_tb['age']/10)

0      4.0
1      3.0
2      4.0
3      4.0
4      3.0
      ... 
995    4.0
996    3.0
997    3.0
998    4.0
999    3.0
Name: age, Length: 1000, dtype: float64

In [30]:
customer_tb['age_rank']

0      40.0
1      30.0
2      40.0
3      40.0
4      30.0
       ... 
995    40.0
996    30.0
997    30.0
998    40.0
999    30.0
Name: age_rank, Length: 1000, dtype: category
Categories (7, float64): [20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0]

In [31]:
# 마스터 데이터에 '60 이상'을 추가한다.
customer_tb['age_rank'].cat.add_categories(['60 이상'], inplace=True)

In [32]:
# 아래 Categories를 보면 '60 이상' 이라는 카테고리 추가됨.
customer_tb['age_rank']

0      40
1      30
2      40
3      40
4      30
       ..
995    40
996    30
997    30
998    40
999    30
Name: age_rank, Length: 1000, dtype: category
Categories (8, object): [20, 30, 40, 50, 60, 70, 80, 60 이상]

In [43]:
# 집약할 데이터를 변경한다.
# 범주형은 = 또는 !=만 판정할 수 있어 isin 함수 이용.
customer_tb.loc[customer_tb['age_rank'].isin([60.0, 70.0, 80.0]), 'age_rank']='60 이상'

In [46]:
customer_tb['age_rank'][:10]

0       40
1       30
2       40
3       40
4       30
5       50
6       50
7    60 이상
8       30
9       30
Name: age_rank, dtype: category
Categories (8, object): [20, 30, 40, 50, 60, 70, 80, 60 이상]

In [47]:
# 사용되지 않는 마스터 데이터를 제거한다.
customer_tb['age_rank'].cat.remove_unused_categories(inplace=True)

In [50]:
# 기존에 60, 70, 80 카테고리 삭제.
customer_tb['age_rank']

0      40
1      30
2      40
3      40
4      30
       ..
995    40
996    30
997    30
998    40
999    30
Name: age_rank, Length: 1000, dtype: category
Categories (5, object): [20, 30, 40, 50, 60 이상]

-----------------------------------------------------------------------------------------------------
## 9.4 범줏값의 조합
성별과 연령대의 범줏값을 조합하여 '남성 20대', '여성 50대' 등과 같은 새로운 범줏값으로 확장하는 것도 가능. <br>

[ 단점 ]
* 범줏값을 조합하면 더미 변수의 수도 조합한 수에 따라 늘어나게 되어 분석에 필요한 데이터양도 늘어난다.
    * 따라서, 범줏값을 조합할 때는 데이터양을 파악하면서 데이터 종류가 많은 범줏값 조합은 피하는 것이 좋다.

In [60]:
# lambd함수에서 sex와 10세 단위로 구분한 age사이에 _를 추가하여 연결한다.
customer_tb['sex_and_age']=pd.Categorical(\
                                          customer_tb[['sex','age']].apply(lambda x: '{}_{}'.format(x[0], np.floor(x[1]/10)*10),  axis=1 ) \
                                         )

In [62]:
customer_tb['sex_and_age']

0        man_40.0
1        man_30.0
2      woman_40.0
3        man_40.0
4        man_30.0
          ...    
995      man_40.0
996      man_30.0
997    woman_30.0
998    woman_40.0
999      man_30.0
Name: sex_and_age, Length: 1000, dtype: category
Categories (14, object): [man_20.0, man_30.0, man_40.0, man_50.0, ..., woman_50.0, woman_60.0, woman_70.0, woman_80.0]

---------------------------------------------------------------
## 9.5 범주형의 수치화